# Tarea de regularización

Jessenia Piza Londoño, Carlos Sebastián Martínez Vidal y Manuela Acosta Fajardo

Grupo 5

[1] En este ejercicio, se requiere predecir la cantidad de solicitudes recibidas usando las otras variables en el conjunto de datos `college.csv`.

In [2]:
# Cargando los modulos basicos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.decomposition import PCA
from sklearn import feature_selection

from time import time

# Para ser reproducible
np.random.seed(42)

# No muestra warnings
import warnings
warnings.filterwarnings('ignore')

Preparamos el dataset. 

In [3]:
# Importando los datos de college
df = pd.read_csv('data/college.csv', index_col=0).dropna()
df.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
Agnes Scott College,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
Alaska Pacific University,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [4]:
# Convertimos las variables categóricas a dummies
df = pd.get_dummies(df, drop_first=True)

Tomaremos como variable de salida Grad.Rate, y el resto de variables serán los predictores. Por ello, dividimos el dataset de la siguiente manera. 

In [5]:
# Definiendo los predictores y la variable de salida
y = df['Grad.Rate']

# Se elimina la variable respuesta (Grad.Rate) y las columnas con las variables categoricas
X = df.drop('Grad.Rate', axis=1).astype('float64')

X.head()

,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Private_Yes
Abilene Christian University,1660.0,1232.0,721.0,23.0,52.0,2885.0,537.0,7440.0,3300.0,450.0,2200.0,70.0,78.0,18.1,12.0,7041.0,1.0
Adelphi University,2186.0,1924.0,512.0,16.0,29.0,2683.0,1227.0,12280.0,6450.0,750.0,1500.0,29.0,30.0,12.2,16.0,10527.0,1.0
Adrian College,1428.0,1097.0,336.0,22.0,50.0,1036.0,99.0,11250.0,3750.0,400.0,1165.0,53.0,66.0,12.9,30.0,8735.0,1.0
Agnes Scott College,417.0,349.0,137.0,60.0,89.0,510.0,63.0,12960.0,5450.0,450.0,875.0,92.0,97.0,7.7,37.0,19016.0,1.0
Alaska Pacific University,193.0,146.0,55.0,16.0,44.0,249.0,869.0,7560.0,4120.0,800.0,1500.0,76.0,72.0,11.9,2.0,10922.0,1.0


    (a) Divida el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba.

In [6]:
# Conjunto de entrenamiento (75%) y de prueba (25%)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state = 42)

# Estandarizamos el conjunto de entrenamiento 
scaler = StandardScaler().fit(X_train)

    (b) Ajuste un modelo lineal usando mínimos cuadrados en el conjunto de entrenamiento e informe el error de prueba (sobre el Test set) obtenido.

Por lo visto en clase, sabemos que mínimos cuadrados equivale a Regresión Ridge cuando $\lambda = 0$.

In [7]:
minimos_cuadrados = Ridge(alpha=0)
minimos_cuadrados.fit(scaler.transform(X_train), y_train) # Ajustamos el modelo a los datos estandarizados
pred = minimos_cuadrados.predict(scaler.transform(X_test)) # Predecimos la salida con el conjunto de Y test 
mean_squared_error(y_test, pred) # MSE

131.36122018103097

El error de prueba obtenido es 131.36122.

Veremos a continuación los coeficientes de los predictores, obtenidos con el método de mínimos cuadrados. 

In [8]:
pd.Series(minimos_cuadrados.coef_, index=X.columns)

Apps           5.379650
Accept        -2.873004
Enroll         2.209390
Top10perc      0.085154
Top25perc      3.678610
F.Undergrad   -1.579863
P.Undergrad   -2.473379
Outstate       2.979427
Room.Board     2.251583
Books         -0.474490
Personal      -1.112617
PhD            1.874837
Terminal      -1.390549
S.F.Ratio      0.142993
perc.alumni    3.653565
Expend        -2.214409
Private_Yes    1.999227
dtype: float64

    (c) Ajuste un modelo de Ridge regression en el conjunto de entrenamiento, con λ elegido por validación cruzada. Informe el error de prueba obtenido.

In [9]:
# Creamos un amplio rango de valores de alpha
alphas = 10**np.linspace(10,-2,100)

# Creamos el modelo de Regresión Ridge, y lo ajustamos 
ridge_cv = RidgeCV(alphas=alphas, cv=5, scoring='neg_mean_squared_error') 
ridge_cv.fit(scaler.transform(X_train), y_train) # Ajustando al Training set

RidgeCV(alphas=array([1.00000000e+10, 7.56463328e+09, 5.72236766e+09, 4.32876128e+09,
       3.27454916e+09, 2.47707636e+09, 1.87381742e+09, 1.41747416e+09,
       1.07226722e+09, 8.11130831e+08, 6.13590727e+08, 4.64158883e+08,
       3.51119173e+08, 2.65608778e+08, 2.00923300e+08, 1.51991108e+08,
       1.14975700e+08, 8.69749003e+07, 6.57933225e+07, 4.97702356e+07,
       3.76493581e+07, 2.84803587e+0...
       6.13590727e+00, 4.64158883e+00, 3.51119173e+00, 2.65608778e+00,
       2.00923300e+00, 1.51991108e+00, 1.14975700e+00, 8.69749003e-01,
       6.57933225e-01, 4.97702356e-01, 3.76493581e-01, 2.84803587e-01,
       2.15443469e-01, 1.62975083e-01, 1.23284674e-01, 9.32603347e-02,
       7.05480231e-02, 5.33669923e-02, 4.03701726e-02, 3.05385551e-02,
       2.31012970e-02, 1.74752840e-02, 1.32194115e-02, 1.00000000e-02]),
        cv=5, scoring='neg_mean_squared_error')

In [10]:
# Mejor alpha 
ridge_cv.alpha_

75.64633275546291

En este caso, el valor seleccionado por medio de validación cruzada es $\lambda=75.6463$.

In [11]:
# Predecimos la salida sobre el conjunto de test, usando el alpha seleccionado
ridge = Ridge(alpha=ridge_cv.alpha_)
ridge.fit(scaler.transform(X_train), y_train)
mean_squared_error(y_test, ridge.predict(scaler.transform(X_test))) # MSE

129.9755886581944

El error de prueba obtenido es 129.9756.

    (d) Ajustar un modelo The Lasso en el conjunto de entrenamiento, con λ elegido por validación cruzada. Informe el error de prueba obtenido, junto con el número de estimaciones de coeficientes distintos de cero.

In [12]:
# Creando los valores de alpha
alphas = 10**np.linspace(10,-2,100)

# Creamos el modelo Lasso, y lo ajustamos 
lassocv = LassoCV(alphas=alphas, cv=10, max_iter=10000)
lassocv.fit(scaler.transform(X_train), y_train)

LassoCV(alphas=array([1.00000000e+10, 7.56463328e+09, 5.72236766e+09, 4.32876128e+09,
       3.27454916e+09, 2.47707636e+09, 1.87381742e+09, 1.41747416e+09,
       1.07226722e+09, 8.11130831e+08, 6.13590727e+08, 4.64158883e+08,
       3.51119173e+08, 2.65608778e+08, 2.00923300e+08, 1.51991108e+08,
       1.14975700e+08, 8.69749003e+07, 6.57933225e+07, 4.97702356e+07,
       3.76493581e+07, 2.84803587e+0...
       6.13590727e+00, 4.64158883e+00, 3.51119173e+00, 2.65608778e+00,
       2.00923300e+00, 1.51991108e+00, 1.14975700e+00, 8.69749003e-01,
       6.57933225e-01, 4.97702356e-01, 3.76493581e-01, 2.84803587e-01,
       2.15443469e-01, 1.62975083e-01, 1.23284674e-01, 9.32603347e-02,
       7.05480231e-02, 5.33669923e-02, 4.03701726e-02, 3.05385551e-02,
       2.31012970e-02, 1.74752840e-02, 1.32194115e-02, 1.00000000e-02]),
        cv=10, max_iter=10000)

In [13]:
# Mejor valor de alpha
lassocv.alpha_

0.2848035868435805

En este caso, el valor seleccionado por medio de validación cruzada es $\lambda=0.2848$.

In [14]:
# Predecimos la salida sobre el conjunto de test, usando el alpha seleccionado
lasso = Lasso(alpha=lassocv.alpha_)
lasso.fit(scaler.transform(X_train), y_train)
mean_squared_error(y_test, lasso.predict(scaler.transform(X_test))) #MSE

131.90633624478227

El error de prueba obtenido es 131.9063.

In [15]:
# Veamos los coeficientes 
pd.Series(lasso.coef_, index=X.columns)

Apps           2.542917
Accept         0.000000
Enroll         0.000000
Top10perc      0.044584
Top25perc      3.718076
F.Undergrad   -0.000000
P.Undergrad   -2.363158
Outstate       2.469199
Room.Board     2.029848
Books         -0.304256
Personal      -0.999071
PhD            0.346463
Terminal      -0.000000
S.F.Ratio      0.000000
perc.alumni    3.526476
Expend        -0.973119
Private_Yes    1.622413
dtype: float64

Podemos observar que el número de estimaciones con coeficientes distintos a cero es **12**. Se puede evidenciar en la tabla anterior.

    (e) Comente los resultados obtenidos. ¿Con qué precisión podemos predecir la cantidad de solicitudes recibidas en la universidad? ¿Hay mucha diferencia entre los errores de prueba resultantes de estos cuatros enfoques?

Teniendo en cuenta los errores de prueba de los modelos utilizados, podemos ver que no hay gran diferencia. Sin embargo, observamos que el modelo que arroja mejores resultados, es decir que tiene un menor error, es el modelo Ridge con $\lambda$ seleccionado con validación cruzada. 

[2] Cargue el dataset de juguete `sklearn.datasets.load_digits(n_class=10, return_X_y=False)`. 

    (a) Divida el conjunto de datos, en Training, Test. Realice el proceso de clasificación usando un clasificador KNN, y determinando el mejor número de vecinos y de componentes principales. Evalue el desempeño en el Test set.


Evalue el desempeño usando la matriz de confusión y el balanced accuracy.

In [40]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

In [54]:
data = load_digits(n_class=10, return_X_y=False)

# Conjunto de entrenamiento (75%) y de prueba (25%)
X_train, X_test, y_train, y_test = train_test_split(data.data,data.target,test_size=0.25, random_state = 42)

# Estandarizamos el conjunto de entrenamiento 
scaler = StandardScaler().fit(X_train)

pca = PCA() # Se instancia el objeto PCA
X_reduced = pca.fit_transform(scaler.transform(X_train)) # Se ajusta y transforma

mean_sqrtknn, mean_sqrtpca = [], []

In [55]:
for i in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(scaler.transform(X_train), y_train)
    mean_sqrtknn.append(mean_squared_error(y_test, knn.predict(scaler.transform(X_test)))) #MSE

In [56]:
n = mean_sqrtknn.index(min(mean_sqrtknn)) + 1

knn2 = KNeighborsClassifier(n_neighbors=n)
knn2.fit(scaler.transform(X_train), y_train)

for q in range(1, 21):        
    X_reduced_test = pca.transform(scaler.transform(X_test))[:,:q]

    knn2.fit(X_reduced[:,:q], y_train)

    # Prediccion con el conjunto de prueba
    pred = knn2.predict(X_reduced_test)
    mean_sqrtpca.append(mean_squared_error(y_test, pred))

In [64]:
print(mean_sqrtknn.index(min(mean_sqrtknn)) + 1, min(mean_sqrtknn))

5 0.33111111111111113


In [65]:
print(mean_sqrtpca.index(min(mean_sqrtpca)) + 1, min(mean_sqrtpca))

18 0.43777777777777777


Note que el mejor número de vecinos es 5 y el mejor número de componentes principales es 18.